# Import Modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

In [2]:
# Import the data
df = pd.read_csv('../RVF_ATX_PID_HZ-2020-07.tsv', sep='\t')
# filter out certain columns
df = df.drop(columns=['local_date', 'local_hour', 'gender', 'age', 'full_panel_reweighted_sag_score', 'home_zip'])
df.head()

,persistentid,venueid,utc_date,utc_hour,dwell
0,5903d26cdcecbd13590c8fe594de785f19b16004e19156...,40b52f80f964a52051001fe3,2020-07-18,22,60
1,cc371dcb888f9ec0ab9a4ecabc4d49e51288313fb17a47...,40b52f80f964a52051001fe3,2020-07-18,22,1011
2,21e09ed692d56697e3c26b777a53a411cd21bed5527c5b...,40b52f80f964a52051001fe3,2020-07-25,0,1058
3,e170f93db0ea4930ea2c0d2167feddb1b4fe2b5412d826...,40b52f80f964a52051001fe3,2020-07-25,18,30
4,63164c43b459b4260a338979948ead113855ddaafb9d52...,40b52f80f964a52051001fe3,2020-07-29,18,763


In [4]:
# filter out rows that are not within a range

# Define the date range
start_date = pd.to_datetime('2020-07-06')
end_date = pd.to_datetime('2020-07-06')
dwell_time_threshold = 60 # in minutes

# Filter rows within the date range
filtered_df = df[(pd.to_datetime(df['utc_date']) >= start_date) & (pd.to_datetime(df['utc_date']) <= end_date) & (df['dwell'] >= dwell_time_threshold)]
# add a new column to dataframe that represents minutes
filtered_df['formatted_utc_hour'] = filtered_df['utc_hour'].astype(int)
filtered_df['arrival_time'] = pd.to_datetime(filtered_df['utc_date']) + pd.to_timedelta(filtered_df['formatted_utc_hour'], unit='h')

filtered_df['departure_time'] = filtered_df['arrival_time'] + pd.to_timedelta(filtered_df['dwell'], unit='m')
filtered_df.head()

/var/folders/pl/lzj1lm8n7v911pypcq90dl8m0000gn/T/ipykernel_98547/3229085348.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['formatted_utc_hour'] = filtered_df['utc_hour'].astype(int)
/var/folders/pl/lzj1lm8n7v911pypcq90dl8m0000gn/T/ipykernel_98547/3229085348.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['arrival_time'] = pd.to_datetime(filtered_df['utc_date']) + pd.to_timedelta(filtered_df['formatted_utc_hour'], unit='h')
/var/folders/pl/lzj1lm8n7v911pypcq90dl8m0000gn/T/

,persistentid,venueid,utc_date,utc_hour,dwell,formatted_utc_hour,arrival_time,departure_time
111,b1e3bf19324b4723450feaf2b874c7de98258a513c084b...,4a48eef9f964a520d7aa1fe3,2020-07-06,18,237795,18,2020-07-06 18:00:00,2020-12-18 21:15:00
169,6b0a0107de38f2a6bd27c8dbbada9d6b459da09115b56c...,509de5c0e4b0a0263e828614,2020-07-06,2,620,2,2020-07-06 02:00:00,2020-07-06 12:20:00
194,75142f982712bcb40c123308c3be6effc57aa387c65958...,509de5c0e4b0a0263e828614,2020-07-06,19,619,19,2020-07-06 19:00:00,2020-07-07 05:19:00
195,afa92b06a82b25b86e8a7c79b0d4ae5c1d8a5036034ab4...,509de5c0e4b0a0263e828614,2020-07-06,22,1085,22,2020-07-06 22:00:00,2020-07-07 16:05:00
233,4579c6d2b23f326d61ef4afefb11e4b4f3a9d1acbf9acc...,50a291687e27e1bf708b80f5,2020-07-06,20,2051,20,2020-07-06 20:00:00,2020-07-08 06:11:00


In [5]:
# add all devices as nodes to Graph
# get all devices
devices = filtered_df['persistentid'].unique()
G = nx.Graph()
for device in devices:
    # add device as a node to G
    G.add_node(device)
    

In [6]:
# # sort the dataframe by arrival time
# filtered_df = filtered_df.sort_values(by=['arrival_time'])
# # reset the index
# filtered_df = filtered_df.reset_index(drop=True)
# filtered_df.head()
# for i, row_i in filtered_df.iterrows():
#     print(f"Row: {i}")
#     for j, row_j in filtered_df.iloc[i+1:].iterrows():
#         if i != j:
#             # check if the two rows are within 30 minutes of each other
#             first = (row_i['arrival_time'], row_i['departure_time'])
#             second = (row_j['arrival_time'], row_j['departure_time'])
#             # check if the two time intervals overlap, by atleast 60 minutes
#             difference = second[0] - first[1]
#             if difference.seconds >= 3600:
#                 # add edge between the two nodes
#                 G.add_edge(row_i['persistentid'], row_j['persistentid'])
# nx.write_gpickle(G, "2020-07-01__to__2020-07-05.gpickle")

In [7]:
# sort the dataframe by arrival time
filtered_df = filtered_df.sort_values(by=['arrival_time'])
# reset the index
filtered_df = filtered_df.reset_index(drop=True)
filtered_df.head()
hashmap = {}
for row in filtered_df.itertuples():
    persistentid = 1
    venueId = 2
    arrival_time = 7
    departure_time = 8
    device = row[persistentid]
    venue = row[venueId]
    timespan = (row[arrival_time], row[departure_time])
    if venue in hashmap:
        hashmap[venue].append((device, timespan))
    else:
        hashmap[venue] = [(device, timespan)]

for location in hashmap:
    # iterate through its list
    li = hashmap[location]
    count = 0
    for item1 in li:
        for item2 in li[count + 1:]:
            if item1[0] != item2[0]:
                # check if the two time intervals overlap, by atleast 15 minutes
                difference = item2[1][0] - item1[1][1]
                if difference.seconds >= 900:
                    # add edge between the two nodes
                    G.add_edge(item1[0], item2[0])
        count += 1
nx.write_gexf(G, "2020-07-06__to__2020-07-06__15-overlap__hashmap.gexf")

In [34]:
# sort the dataframe by arrival time
filtered_df = filtered_df.sort_values(by=['arrival_time'])
# reset the index
filtered_df = filtered_df.reset_index(drop=True)
filtered_df.head()
count = 0
for row_i in filtered_df.itertuples():
    if count % 1000 == 0:
        print(f"Row: {count}")
    for row_j in filtered_df.iloc[count+1:].itertuples():
        persistentid = 1
        venueId = 2
        arrival_time = 7
        departure_time = 8
        if row_i[persistentid] != row_j[persistentid] and row_i[venueId] == row_j[venueId]:
            # check if the two rows are within 30 minutes of each other
            first = (row_i[arrival_time], row_i[departure_time])
            second = (row_j[arrival_time], row_j[departure_time])
            # check if the two time intervals overlap, by atleast 60 minutes
            difference = second[0] - first[1]
            if difference.seconds >= 3600:
                # add edge between the two nodes
                G.add_edge(row_i[persistentid], row_j[persistentid])
    count += 1
nx.write_gexf(G, "2020-07-01__to__2020-07-05.gexf")

Row: 0
Row: 1000
Row: 2000
Row: 3000
Row: 4000
Row: 5000
Row: 6000


KeyboardInterrupt: 

In [40]:
# get stats about the graph
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")
print(f"Number of connected components: {nx.number_connected_components(G)}")
print(f"Average clustering coefficient: {nx.average_clustering(G)}")
print(f"Average shortest path length: {nx.average_shortest_path_length(G)}")
print(f"Average degree: {np.mean([val for (node, val) in G.degree()])}")
print(f"Average degree centrality: {np.mean(list(nx.degree_centrality(G).values()))}")
print(f"Average closeness centrality: {np.mean(list(nx.closeness_centrality(G).values()))}")
print(f"Average betweenness centrality: {np.mean(list(nx.betweenness_centrality(G).values()))}")
print(f"Average eigenvector centrality: {np.mean(list(nx.eigenvector_centrality(G).values()))}")
print(f"Average pagerank: {np.mean(list(nx.pagerank(G).values()))}")

Number of nodes: 6935
Number of edges: 36095
Number of connected components: 963
Average clustering coefficient: 0.5222032583154474


NetworkXError: Graph is not connected.

In [78]:
# # connecting edges between every device if they were at the same location for a specified amount of dwell time
# groups = dict()
# # df['combined_data'] = df['venueid'].astype(str) + df['utc_date'].astype(str) + df['utc_hour'].astype(str)
# filtered_df['index'] = filtered_df.index
# grouped_df = filtered_df.groupby(['venueid', 'utc_date', 'utc_hour'])['index'].apply(list).reset_index(name='group')
# grouped_df.head()

In [77]:
# for e in grouped_df['group']:
#     if len(e) > 1:
#         for n1 in e:
#             for n2 in e:
#                 if n1 == n2: continue
#                 G.add_edge(n1, n2)

In [28]:
# nx.draw(G, with_labels=True)